# Program Collections generator

This script parses a xlsx file containing metadata about Program Collections and searches the NIAID Data Ecosystem Discovery Portal for records that should be included as part of each program. It then generates the corrections files needed to add the program collection to the Discovery Portal

In [ ]:
import os
import requests
import json
import pandas as pd

In [ ]:
script_path = os.getcwd()
data_path = os.path.join(script_path,'data')
file_path = os.path.join(data_path,'Program Collections.xlsx')
p1_path = os.path.abspath(os.path.join(script_path, os.pardir))
parent_path = os.path.abspath(os.path.join(p1_path, os.pardir))
correction_path = os.path.join(parent_path,'nde-metadata-corrections')
print(correction_path)

In [ ]:
#approvedProd = ['GCID','HIPC','CREID']
approvedProd = []
with open(os.path.join(data_path,'approved_for_prod.txt'),'r') as prodinfile:
    for line in prodinfile:
        approvedProd.append(line.strip())
print(approvedProd)

In [ ]:
def parse_array_text(arraytext):
    if '*' in arraytext:
        arraytext = arraytext.replace('*','')
    if ',' in arraytext:
        arrayinfo = arraytext.split(',')
        cleanarray = [x.strip() for x in arrayinfo]
    elif '|' in arraytext:
        arrayinfo = arraytext.split('|')
        cleanarray = [x.strip() for x in arrayinfo]
    else:
        cleanarray = [arraytext]
    return cleanarray

def generate_meta_file(correction_path, approvedProd, row):
    clean_descript = f"{row['description']} For more information, visit the NIAID program page: {row['niaidURL']}"
    altname = parse_array_text(row['alternateName'])
    parentorgs = parse_array_text(row['parentOrganization']) 
    meta_dict = {"@type": "ResearchProject", "name":row["name"], "abstract":row["abstract"],
                 "description":clean_descript, "alternateName":altname,
                 "url": row["url"], "parentOrganization":parentorgs} 
    file_name = f'{row["fileName"]}_correction.json'
    output_dict = {"sourceOrganization":[meta_dict]}
    if row["fileName"] in approvedProd:
        outputpath = os.path.join(correction_path,"collections_corrections_production")
    else:
        outputpath = os.path.join(correction_path,"collections_corrections_staging")
    with open(os.path.join(outputpath,file_name),'w') as outwrite:
        outwrite.write(json.dumps(output_dict,indent=4))

In [ ]:
def clean_program_list(base_meta):
    tmpdf = base_meta.loc[base_meta['fundingIDList']!='not found']
    cleandf = tmpdf.loc[tmpdf['fileName']!='--'].copy()
    return cleandf    

In [ ]:
base_meta = pd.read_excel(file_path, 'metadata', engine='openpyxl')
clean_meta = clean_program_list(base_meta)
print(clean_meta.tail(3))
test_meta = base_meta.loc[~base_meta['fundingIDList'].isna() & ~base_meta['niaidURL'].isna()]
print(test_meta.head(n=5))

In [ ]:
test_row = test_meta.iloc[1]
generate_meta_file(correction_path, approvedProd, test_row)

In [ ]:
activity_codes_df = pd.read_csv(os.path.join(data_path,'NIH_IC_codes.tsv'),delimiter='\t',header=0)
print(activity_codes_df.head(n=2))

In [ ]:
def load_codes(data_path):
    activity_codes_df = pd.read_csv(os.path.join(data_path,'NIH_activity_codes.csv'),delimiter=',',header=0)
    tmp_codes = activity_codes_df['Activity Code'].unique().tolist()
    act_codes = [x.strip() for x in tmp_codes]
    ic_codes_df = pd.read_csv(os.path.join(data_path,'NIH_IC_codes.tsv'),delimiter='\t',header=0)
    temp_codes = ic_codes_df['Code'].unique().tolist()
    ic_codes = [x.strip() for x in temp_codes]
    return act_codes, ic_codes

def check_grantID_start(grantID, act_codes, ic_codes):
    first_char = grantID[0]
    first_two = grantID[0:2]
    try:
        int(first_two)
        idstart = "contract"
    except:
        try:
            int(first_char)
            idstart = "application_type"
        except:
            two_char = grantID[0:2]
            if '-' in grantID[0:3]:
                three_char = grantID[0:4].replace('-','')
            elif ' ' in grantID[0:3]:
                three_char = grantID[0:4].replace(' ','')
            else:
                three_char = grantID[0:3]
            if two_char in ic_codes:
                idstart = "ic_code"
            elif three_char in act_codes:
                idstart = "activity_code"
            else:
                idstart = "unknown"
    return idstart

def check_grantID_end(grantID):
    if '-' in grantID[-5:]:
        idend = "FY"
    else:
        idend = "project_code"
    return idend

def parse_apptype_start(grantID): ## Eg. 1-R01 vs 1R01 vs 1 RO1
    applTypeCode = str(grantID[0])
    if '-' in grantID[0:2]:
        remaining_grantID = grantID[2:]
    elif ' ' in grantID[0:2]:
        remaining_grantID = grantID[2:]
    else:
        remaining_grantID = grantID[1:]
    return applTypeCode, remaining_grantID

def parse_actcode_start(grantID):
    if '-' in grantID[0:3]: ## Eg. U-01 vs U01 vs U-01- vs U01-
        activityCode = grantID[0:4].replace('-','')
        if '-' in grantID[4:7]:
            remaining_grantID = grantID[5:]
        else:
            remaining_grantID = grantID[4:]
    else:
        activityCode = grantID[0:3]
        if '-' in grantID[3:6]:
            remaining_grantID = grantID[4:]
        else:
            remaining_grantID = grantID[3:]
    return activityCode, remaining_grantID

def parse_iccode_start(grantID):
    icCode = grantID[0:2]
    if '-' in grantID[2:5]: ## Eg. AI089992-11, AI-089992-11
        remaining_grantID = grantID[3:]
    else:
        remaining_grantID = grantID[2:]
    return icCode, remaining_grantID

def parse_serial_start(grantID):
    if len(grantID) > 6:
        serialNum = grantID[0:6]
        remaining_grantID = grantID[7:]
    else:
        serialNum = grantID
        remaining_grantID = ''
    return serialNum, remaining_grantID

def parse_grantID(grantID, act_codes, ic_codes):
    idstart = check_grantID_start(grantID, act_codes, ic_codes)
    idend = check_grantID_end(grantID)
    if idend == "FY":
        supportYear = grantID[-2:]
    else:
        supportYear = "not found"
    if idstart == "application_type":
        applTypeCode, grantID_child0 = parse_apptype_start(grantID)
        activityCode, grantID_child1 = parse_actcode_start(grantID_child0)
        icCode, grantID_child2 = parse_iccode_start(grantID_child1)
        serialNum, remaining_grantID = parse_serial_start(grantID_child2)
    elif idstart == "activity_code":
        applTypeCode = "not found"
        activityCode, grantID_child1 = parse_actcode_start(grantID)
        icCode, grantID_child2 = parse_iccode_start(grantID_child1)
        serialNum, remaining_grantID = parse_serial_start(grantID_child2)
    elif idstart == "ic_code":
        applTypeCode = "not found"
        activityCode = "not found"
        icCode, grantID_child2 = parse_iccode_start(grantID)
        serialNum, remaining_grantID = parse_serial_start(grantID_child2)      
    else:
        applTypeCode = "not found"
        activityCode = "not found"
        icCode = "not found"
        serialNum = "not found"
    grantObject = {"grantID":grantID,
                   "applTypeCode":applTypeCode, 
                   "activityCode":activityCode, 
                   "icCode":icCode, 
                   "serialNum":serialNum, 
                   "supportYear":supportYear}
    return grantObject

grantlist = parse_array_text(test_row['fundingIDList'])
print(grantlist[0])

In [ ]:
grantIDList = ["1-R01-AI073685-01","1R01-AI073685-01","1-R01AI073685-01","1-R-01AI073685-01","R01AI073685-01","R01AI073685-01","R01-AI073685-01","AI073685-01","1-R-01-AI073685-01", "AI073685"]
act_codes, ic_codes = load_codes(data_path)
for eachgrant in grantIDList:
    grantObject = parse_grantID(eachgrant, act_codes, ic_codes)
    print(grantObject)
    

### Searching for records related to grant IDs:

To do:
- Pull the datasets associated with the differently formatted grantIDs
- Compare the results to identify the best approach for pulling records based on grantIDs
  - It looks like the combined icCode+project number may yield the most results

In [ ]:
def search_for_records(grantlist):
    resultlist = []
    faillist = []
    for eachgrant in grantlist:
        #print(eachgrant)
        r = requests.get(f"https://api-staging.data.niaid.nih.gov/v1/query?&q=funding.identifier:*{eachgrant}*&fields=_id,funding.identifier&size=500")
        s = requests.get(f"https://api-staging.data.niaid.nih.gov/v1/query?&q=funding.identifier:{eachgrant}&fields=_id,funding.identifier&size=500")
        temp = json.loads(r.text)
        temp2 = json.loads(s.text)
        try:
            temphits = len(temp['hits'])
        except:
            temphits = 0
        try:
            temphits2 = len(temp['hits'])
        except:
            temphits2 = 0
        if temphits > 0:
            if temphits2 > 0:
                alltemp = temp['hits'] + temp2['hits']
            else:
                alltemp = temp['hits']
        elif temphits2 > 0:
                alltemp = temp2['hits']
        else:
            faillist.append(eachgrant)
            alltemp = []
        if len(alltemp)>0:
            for eachhit in alltemp:
                tmpid = eachhit['_id']
                if isinstance(eachhit['funding'],list):
                    for eachfunding in eachhit['funding']:
                        if eachgrant in eachfunding['identifier']:
                            resultlist.append({"query":eachgrant,"_id":tmpid,"fundID":eachfunding['identifier']})
                elif isinstance(eachhit['funding'],dict):
                    resultlist.append({"query":eachgrant,"_id":tmpid,"fundID":eachhit['funding']['identifier']})
    resultdf = pd.DataFrame(resultlist)
    clean_result = resultdf.drop_duplicates(keep="first")
    print(faillist)
    return clean_result

In [ ]:
grantIDList = ["1-R01-AI073685-01","1R01-AI073685-01","1-R01AI073685-01","1-R-01AI073685-01","R01AI073685","R01AI073685-01","R01 AI073685-01", "R01 AI07368501","R01-AI073685-01","AI073685-01","1-R-01-AI073685-01", "AI073685"]
clean_result = search_for_records(grantIDList)

In [ ]:
print(clean_result)

In [ ]:
## parse the funder ids from the spreadsheet and format into icCode+project number
def parse_program_funding(fundingInfo):
    grantlist = []
    temp = parse_array_text(fundingInfo)
    temp2 = [x.strip() for x in temp]
    temp3 = [x.replace("*","") for x in temp2]
    for eachgrant in temp3:
        try:
            grantObject = parse_grantID(eachgrant, act_codes, ic_codes)
            if grantObject['icCode']!='not found' and grantObject['serialNum'] != 'not found':
                grantlist.append(grantObject['icCode']+grantObject['serialNum'])
            else:
                grantlist.append(eachgrant)
        except:
            grantlist.append(eachgrant)
    return grantlist

In [ ]:
def parse_prior_program(priorfunding):
    if priorfunding != "not found":
        tmp = priorfunding.split("|")
        priorgrants = [x.strip() for x in tmp]
    else:
        priorgrants = []
    return priorgrants

In [ ]:
def generate_inclusion_list(correction_path,approvedProd,row):
    filename = f"{row['fileName']}_records.txt"
    fundingInfo = row['fundingIDList']
    priorfunding = row['PriorProjectGrantIDs']
    basegrantlist = parse_program_funding(fundingInfo)
    if len(basegrantlist) == 0:
        print(filename, " basegrantlist is 0")
    priorgrantlist = parse_prior_program(priorfunding)
    if len(priorgrantlist)>0:
        #print(priorgrantlist)
        grantlist = list(set(basegrantlist).union(set(priorgrantlist)))
    else:
        grantlist = list(set(basegrantlist))
    clean_result = search_for_records(grantlist)
    print(clean_result)
    if row["fileName"] in approvedProd:
        outputpath = os.path.join(correction_path,"collections_corrections_production")
    else:
        outputpath = os.path.join(correction_path,"collections_corrections_staging")
    with open(os.path.join(outputpath,filename),'w') as outwrite:
        if len(clean_result)>0:
            for eachrecord in clean_result['_id'].unique().tolist():
                outwrite.write(f'https://data.niaid.nih.gov/resources?id={eachrecord}\n')
        else:
            outwrite.write("")

In [ ]:
#print(len(test_meta))
#print(test_meta.iloc[2]['fundingIDList'])
#print(test_meta.head(n=3))
#generate_inclusion_list(test_meta.iloc[2])
#print(test_meta.head(n=2))
#test_meta.apply(lambda row: generate_meta_file(correction_path, approvedProd, row), axis=1)
#test_meta.apply(lambda row: generate_inclusion_list(correction_path,approvedProd,row), axis=1)
clean_meta.apply(lambda row: generate_meta_file(correction_path, approvedProd, row), axis=1)
clean_meta.apply(lambda row: generate_inclusion_list(correction_path,approvedProd,row), axis=1)

In [ ]:
eachgrant=grantIDList[-1]
r = requests.get(f"https://api-staging.data.niaid.nih.gov/v1/query?&q=funding.identifier:*{eachgrant}*&fields=_id,funding.identifier&size=500")
temp = json.loads(r.text)
print(temp)

In [ ]:
test_meta = clean_meta.iloc[7]
print(test_meta['fundingIDList'])
print(test_meta['PriorProjectGrantIDs'])

basegrantlist = parse_program_funding(test_meta['fundingIDList'])
priorprogramlist = parse_prior_program(test_meta['PriorProjectGrantIDs'])
print(basegrantlist)
print(priorprogramlist)
if len(priorprogramlist)>0:
    grantlist = list(set(basegrantlist).union(set(priorprogramlist)))
    print(grantlist)